[참조]
- https://wikidocs.net/57165


<이전>
- `torch.utils.data.Dataset`, `torch.utils.data.DataLoader`
- 위의 도구 사용하면 미니 배치 학습, 셔플(suffle), 병렬 처리 간단히 수행 가능 
- 순서) `Dataset` 정의 -> `DataLoader`에 전달 

# 커스텀 데이터셋 (Custom Dataset)
----
- `torch.utils.data.Dataset` 상속받아 직접 커스텀 데이터셋(Custom Dataset) 만드는 경우
- `torch.utils.data.Dataset`: PyTorch에서 제공하는 추상 class


## 1. 커스텀 데이터셋(Custom Dataset)
- `Dataset` 상속받아 method 오버라이드하여 커스텀 데이터셋 만들어보기

```
## 기본적으로 필요한 define 3개
class CustomDataset(torch.utils.data.Dataset):
  # 데이터셋 전처리 해주는 부분
  def__init__(self):  

  # 데이터셋 길이 -> 총 샘플의 수 적어주는 부분
  def __len__(self):
 
  # 데이터셋에서 특징 1개의 샘플 가져오느 함수
  def __getitem__(self, idx):
```
- `len` -> `len(dataset)` 수행 시 데이터셋 크기 리턴
- `get_item` -> `dataset[i]` 수행 시 `i`번째 샘플 가져오도록 인덱싱

In [1]:
# ## 기본적으로 필요한 define 3개
# class CustomDataset(torch.utils.data.Dataset):
#   # 데이터셋 전처리 해주는 부분
#   def__init__(self):  

#   # 데이터셋 길이 -> 총 샘플의 수 적어주는 부분
#   def __len__(self):
 
#   # 데이터셋에서 특징 1개의 샘플 가져오느 함수
#   def __getitem__(self, idx):

## 2. 커스텀 데이터셋으로 선형 회귀 구현 

In [2]:
import torch
import torch.nn.functional as F

In [3]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

`Dataset` 상속

In [4]:
class CustomDataset(Dataset):
  def __init__(self):
    self.x_data = [[73, 80, 75],
                   [93, 88, 93],
                   [89, 91, 90],
                   [96, 98, 100],
                   [73, 66, 70]]
    self.y_data = [[152], [185], [180], [196], [142]]
  
  # 총 데이터의 개수 리턴
  def __len__(self):
    return len(self.x_data)

  # index 입력받아 그에 mapping되는 입출력 데이터 -> PyTorch Tensor 형태로 리턴
  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = torch.FloatTensor(self.y_data[idx])
    return x, y

In [5]:

dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [6]:
model = torch.nn.Linear(3, 1)
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-5)

In [9]:
nb_epochs = 20
for epoch in range(nb_epochs+1):
  for batch_idx, samples in enumerate(dataloader):
    # print(batch_idx)
    # print(samples)
    x_train, y_train = samples

    # H(x)
    prediction = model(x_train)

    # cost
    cost = F.mse_loss(prediction, y_train)

    # cost -> H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} | Batch {}/{} | Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))


Epoch    0/20 | Batch 1/3 | Cost: 0.117260
Epoch    0/20 | Batch 2/3 | Cost: 5.053760
Epoch    0/20 | Batch 3/3 | Cost: 2.265545
Epoch    1/20 | Batch 1/3 | Cost: 2.641428
Epoch    1/20 | Batch 2/3 | Cost: 3.151036
Epoch    1/20 | Batch 3/3 | Cost: 1.135791
Epoch    2/20 | Batch 1/3 | Cost: 0.907828
Epoch    2/20 | Batch 2/3 | Cost: 4.662889
Epoch    2/20 | Batch 3/3 | Cost: 0.051621
Epoch    3/20 | Batch 1/3 | Cost: 3.210944
Epoch    3/20 | Batch 2/3 | Cost: 0.807331
Epoch    3/20 | Batch 3/3 | Cost: 4.466667
Epoch    4/20 | Batch 1/3 | Cost: 4.835881
Epoch    4/20 | Batch 2/3 | Cost: 2.088975
Epoch    4/20 | Batch 3/3 | Cost: 0.001472
Epoch    5/20 | Batch 1/3 | Cost: 3.603772
Epoch    5/20 | Batch 2/3 | Cost: 2.461199
Epoch    5/20 | Batch 3/3 | Cost: 0.331324
Epoch    6/20 | Batch 1/3 | Cost: 2.053687
Epoch    6/20 | Batch 2/3 | Cost: 3.845572
Epoch    6/20 | Batch 3/3 | Cost: 0.072466
Epoch    7/20 | Batch 1/3 | Cost: 1.542694
Epoch    7/20 | Batch 2/3 | Cost: 5.508164
Epoch    7/

In [10]:
## 임의의 값 [73, 80, 75]
pred_y = torch.FloatTensor([[73, 80, 75]])
# 입력값 [73, 80, 75]에 대한 예측값 y return -> pred_y 저장 
pred_y = model(pred_y)

print('pred_y: ', pred_y)

pred_y:  tensor([[152.5399]], grad_fn=<AddmmBackward0>)


In [11]:
## original
# self.x_data = [[73, 80, 75],
#                 [93, 88, 93],
#                 [89, 91, 90],
#                 [96, 98, 100],
#                 [73, 66, 70]]
# self.y_data = [[152], [185], [180], [196], [142]]